## Pokemon types, aka "super effective"

In Pokemon, `attacks` have 1 of 18 [types](https://pokemondb.net/type),
like `fire` or `water`. The `monsters` themselves have single or 
[dual type](https://pokemondb.net/type/dual), eg `fire` or `fire/bug`.
Note that `fire/bug` and `bug/fire` mean the same thing.

Some types are stronger or weaker against others. 
For example, `water` attacks deal 2x damage on `fire` monsters 
and 1/2 against `water`.
For dual-type monsters, attacks apply to each type then multiply.
So `water` deals 4x damage to `fire/ground`
and 1x (regular) damage to `fire/water`.

**Goal**: programatically find the strongest and weakest of the 
18 + 18*17/2 = 171 single or dual types.

Ideas
1. Replicate the [cumulative scores](https://pokemondb.net/type/dual). Types with fewest and most weaknesses. Count number of pokemons of each type.
2. attack type coverage: enumerate all 4-tuples of types that 2x all types
3. input pokemon name, find its single/dual type, recommend moveset maximizing coverage, eg ice monster should carry ground attack to counter rock and steel STABers


In [1]:
water_elec_data = {
    'water': {'water':1/2},
    'electric': {'water':2, 'electric':1/2}
}

full_type_matchup_data = {
    'normal': {
        'rock': 1/2,
        'ghost': 0,
        'steel': 1/2
    },
    'fire': {
        'fire':1/2,
        'water':1/2,
        'grass':2,
        'ice':2,
        'bug':2,
        'rock':1/2,
        'dragon':1/2,
        'steel':2
    },
    'water': {
        'fire':2,
        'water':1/2,
        'grass':1/2,
        'ground':2,
        'rock':2,
        'dragon':1/2
    },
    'electric': {
        'water':2,
        'electric':1/2,
        'grass':1/2,
        'ground':0,
        'fly':2,
        'dragon':1/2
    },
    'grass': {
        'fire':1/2,
        'water':2,
        'grass':1/2,
        'poison':1/2,
        'ground':2,
        'fly':1/2,
        'bug':1/2,
        'rock':2,
        'dragon':1/2,
        'steel':1/2
    },
    'ice': {
        'fire':1/2,
        'water':1/2,
        'grass':2,
        'ice':1/2,
        'ground':2,
        'fly':2,
        'dragon':2,
        'steel':2
    },
    'fighting': {
        'normal':2,
        'ice':2,
        'poison':1/2,
        'flying':1/2,
        'bug':1/2,
        'rock':2,
        'ghost':0,
        'drak':2,
        'steel':2,
        'fairy':1/2
    },
    'poison': {
        'grass':2,
        'poison':1/2,
        'ground':1/2,
        'rock':1/2,
        'ghost':1/2,
        'steel':0,
        'fairy':2
    },
    'ground': {
        'fire':2,
        'electric': 2,
        'grass':1/2,
        'poison':2,
        'flying':0,
        'bug':1/2,
        'rock':2,
        'steel':2
    },
    'flying': {
        'electric':1/2,
        'grass':2,
        'fighting':2,
        'bug':2,
        'rock':1/2,
        'steel':1/2
    },
    'psychic': {
        'fighting':2,
        'poison':2,
        'psychic':1/2,
        'dark':0,
        'steel':1/2
    },
    'bug': {
        'fire':1/2,
        'grass':2,
        'fighting':1/2,
        'poison':1/2,
        'flying':1/2,
        'psychic':2,
        'rock':1/2,
        'dark':2,
        'steel':1/2,
        'fairy':1/2
    },
    'rock': {
        'fire':2,
        'ice':2,
        'fighting':1/2,
        'flying':2,
        'bug':2,
        'steel':1/2
    },
    'ghost': {
        'normal':0,
        'psychic':2,
        'ghost':2,
        'dark':1/2
    },
    'dragon': {
        'dragon':2,
        'steel':1/2,
        'fairy':0
    },
    'dark': {
        'fighting':1/2,
        'psychic':2,
        'ghost':2,
        'dark':1/2,
        'fairy':1/2
    },
    'steel': {
        'fire':1/2,
        'water':1/2,
        'electric':1/2,
        'ice':2,
        'rock':2,
        'steel':1/2,
        'fairy':2
    },
    'fairy': {
        'fire':1/2,
        'fighting':2,
        'poison':1/2,
        'dragon':2,
        'dark':2,
        'steel':1/2
    }
}


In [2]:
from collections import defaultdict


def test_build_effectiveness(f):
    d_in = {
        'fire':{'water':1/2}, 
        'water':{'fire':2},
    }
    d_out = {
        'fire':{'fire':1,'water':2},
        'water':{'fire':1/2,'water':1},
        ('fire','water'):{'fire':1/2,'water':2}
    }
    assert dict(f(d_in)) == d_out
    print('test_build_effectiveness passed')
    
    
def build_effectiveness(data):
    """ data = map of 171 defense elements -> 18 atk elements -> effectiveness
    """
    # map each type or dual type to a dict of type effectiveness
    resist = defaultdict(dict) # map single/dual type -> atk type -> effectiveness
    all_types = data.keys()
    
    for atk_t in all_types:
        for t1 in all_types:
            # single type
            eff = data[atk_t][t1] if t1 in data[atk_t] else 1        
            resist[t1][atk_t] = eff
            # dual types
            for t2 in all_types:
                if t1 >= t2: # t1,t2 is the same as t2,t1. Keep the first alphabetically.
                    continue
                eff = (
                    (data[atk_t][t1] if t1 in data[atk_t] else 1)
                    * (data[atk_t][t2] if t2 in data[atk_t] else 1)
                )
                resist[(t1,t2)][atk_t] = eff
    return resist

test_build_effectiveness(build_effectiveness)

effectiveness_data = build_effectiveness(full_type_matchup_data)


test_build_effectiveness passed


In [3]:
len(effectiveness_data) # 171 for all single+dual types

171

In [4]:
# count number of atk types strong/weak/reg vs a given pkmn type (single or dual)

def get_pkmn_type_matchup(effective_data, pokemon_type):
    """ return number of atk types resisted by this pokemon type
    effective_data: map each pokemon type to a map of the effectiveness 
    of each atk type on that pokemon type. eg {fire: {water:2, fire:1/2}, water:{}}
    pokemon_type: 'water' or ('water','grass')
    """
    atk_t_dict = effective_data[pokemon_type]
    n_regular, n_weak_to, n_strong_to = 0, 0, 0 # number of atk types
    for atk_t, eff in atk_t_dict.items():
        if eff == 1:
            n_regular += 1
        if eff < 1:
            n_strong_to += 1
        if eff > 1:
            n_weak_to += 1
    return n_regular, n_strong_to, n_weak_to


def test_get_pkmn_type_matchup(f):
    effective_data = {
        'electric': {'water': 1, 'electric': 1/2},
        'water': {'water': 1/2, 'electric': 2},
        ('electric','water'): {'water': 1/2, 'electric': 1}
    }
    assert f(effective_data, 'water') == (0, 1, 1)
    print('test_get_pkmn_type_matchup passed')
    
    
test_get_pkmn_type_matchup(get_pkmn_type_matchup)


test_get_pkmn_type_matchup passed


In [5]:
# part 2: find `n_atk_t` attack types covering highest number of single+dual pkmn types

def generate_atk_type_combinations(all_types, n_atk):
    """ return all possible combinations C(n_types, n_atk) 
    of n_atk attack types among all_types.
    all_types: array of types str
    n_atk: can be 2, 3, or 4 
    return: list of n_atk-size tuples
    """
    type_combinations = []
    
    for t1 in all_types:
        
        for t2 in all_types:
            if t2 <= t1:
                continue # ignore duplicates: a,b == b,a
            if n_atk == 2:
                type_combinations.append((t1,t2))
                continue # don't go into logic for 3+ types
                
            for t3 in all_types:
                if t3 <= t2 or t3 <= t1:
                    continue # ignore duplicates
                if n_atk == 3:
                    type_combinations.append((t1,t2,t3))
                    continue # ignore 4-type logic

                for t4 in all_types:
                    if t4 <= t3 or t4 <= t2 or t4 <= t1:
                        continue # ignore dupes
                    type_combinations.append((t1,t2,t3,t4))
                    
    return type_combinations
    
    
def compute_coverage(atk_types, effectiveness_data):
    """
    Compute number of pokemon types strong/weak/regular 
    to an attacker with moves of 2-4 types.
    effectiveness_data: dict mapping (single or dual) pokemon type 
    to the effectiveness of each attack type on that pokemon.
    atk_types: a 2- 3- or 4-tuple of attack types
    return: a 3-tuple of number of pkmn types weak/strong/reg to atk_types
    """
    n_atk_types = len(atk_types)
    
    # these will sum to len(effectiveness_data)
    n_covered, n_resisting, n_regular = 0, 0, 0 
    
    for pkmn_t, atk_t_data in effectiveness_data.items():
        if (
            atk_t_data[atk_types[0]] > 1 
            or atk_t_data[atk_types[1]] > 1 
            or (n_atk_types >= 3 and atk_t_data[atk_types[2]] > 1)
            or (n_atk_types == 4 and atk_t_data[atk_types[3]] > 1)
        ):
            n_covered += 1 # TODO: weigh by number of pokemon of this type
            
        elif (
            atk_t_data[atk_types[0]] < 1 
            and atk_t_data[atk_types[1]] < 1 
            and (n_atk_types >= 3 and atk_t_data[atk_types[2]] < 1)
            and (n_atk_types == 4 and atk_t_data[atk_types[3]] < 1)
        ):
            n_resisting += 1 
        else:
            n_regular += 1
            
    return n_covered, n_resisting, n_regular

    
def build_atk_types_coverages(full_type_matchup_data, n_atk=3):
    """
    algo:
    1. enumerate all combinations of n_atk=3 or 4 attack types: 
    C(18,3)=816 and C(18,4)=3060
    2. for each combination tuple, compute number of pkmn types (single+dual) who:
    2a: are weak against at least 1 atk type --> covered
    2b. resist all 4 atk types --> resisting
    2c. otherwise --> regular (so, reg + resist + covered = 171)
    3. sort the tuples by `covered` ascendingly then `resisting` descendingly
    """

    all_types = full_type_matchup_data.keys()
    # map pkmn_t to atk_t to effectivness in (0,1/4,1/2,1,2,4)
    effectiveness_data = build_effectiveness(full_type_matchup_data)
    coverages = {} 
    
    all_atk_type_combinations = generate_atk_type_combinations(all_types, n_atk)
    for type_combination in all_atk_type_combinations:
        n_cov, n_res, n_reg = compute_coverage(type_combination, effectiveness_data)
        coverages[type_combination] = n_cov, n_res, n_reg
    
    """
    for t1 in all_types:
        for t2 in all_types:
            if t2 <= t1:
                continue # ignore duplicates: a,b == b,a
            for t3 in all_types:
                if t3 <= t2 or t3 <= t1:
                    continue # ignore duplicates
                n_covered, n_resisting, n_regular = 0,0,0
                # iterate over all 171 possible pokemon types
                for pkmn_t, atk_t_data in effectiveness_data.items():
                    if (atk_t_data[t1] > 1 or atk_t_data[t2] > 1 
                        or atk_t_data[t3] > 1):
                        n_covered += 1
                    elif (atk_t_data[t1] < 1 and atk_t_data[t2] < 1 
                          and atk_t_data[t3] < 1):
                        n_resisting += 1 
                    else:
                        n_regular += 1
                # store number of pkmn types weak/strong/reg to these 3 atk types
                # TODO: could weigh by number of pokemons in each pkmn type 
                coverages[(t1,t2,t3)] = n_covered, n_resisting, n_regular
    """
    return coverages


def test_build_atk_types_coverages(f, full_type_matchup_data):
    n_atk_types = 3
    coverages = f(full_type_matchup_data, n_atk_types)
    
    # expect 816 distinct 3-tuples for 18 base types: 18! / 15! / 3!
    # n_base_types = len(full_type_matchup_data)
    # import scipy.special
    # expected_len = scipy.special.comb(n_base_types, n_atk_types, exact=True)
    expected_len = 816
    assert len(coverages) == expected_len
    
    print('test_build_atk_types_coverages passed')
    
    
test_build_atk_types_coverages(build_atk_types_coverages, full_type_matchup_data)

coverages = build_atk_types_coverages(full_type_matchup_data, 3)
print(len(coverages))

test_build_atk_types_coverages passed
816


In [6]:
# sort tuples of atk types by number of pkmn covered asc,
# then by number of pkmn resisting desc

coverage_list = [(atk_tup, counts) for atk_tup, counts in coverages.items()]
coverage_list.sort(key=lambda x: (x[1][0], -x[1][1]), reverse=True)
coverage_list[:20]

[(('ground', 'ice', 'rock'), (137, 0, 34)),
 (('fairy', 'ground', 'rock'), (137, 0, 34)),
 (('fairy', 'fire', 'ground'), (132, 0, 39)),
 (('flying', 'ground', 'rock'), (130, 0, 41)),
 (('bug', 'ground', 'rock'), (129, 0, 42)),
 (('fairy', 'ice', 'rock'), (127, 0, 44)),
 (('ice', 'psychic', 'rock'), (125, 0, 46)),
 (('flying', 'ground', 'ice'), (125, 0, 46)),
 (('flying', 'ground', 'steel'), (125, 0, 46)),
 (('grass', 'ice', 'rock'), (124, 0, 47)),
 (('fairy', 'flying', 'ground'), (124, 0, 47)),
 (('fighting', 'ice', 'rock'), (123, 0, 48)),
 (('ground', 'ice', 'steel'), (123, 0, 48)),
 (('ground', 'poison', 'rock'), (123, 0, 48)),
 (('flying', 'grass', 'ground'), (123, 0, 48)),
 (('fire', 'ground', 'ice'), (122, 0, 49)),
 (('fire', 'ground', 'rock'), (122, 0, 49)),
 (('grass', 'ground', 'rock'), (122, 0, 49)),
 (('ice', 'rock', 'steel'), (122, 0, 49)),
 (('ghost', 'ground', 'rock'), (122, 0, 49))]

In [7]:
# notice: for 3 atk types, ground and rock are in 7 of top 10, and ice in 5.
# that's because ground is strong against 5 basic types, and rock/ice against 4,
# whereas other types are only strong vs 3 or less 
# (except fighting:5, but fight+ground covers 7, fight+rock 7, but ground+rock 8).
